In [10]:
import os
import xraydb
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
from scipy.interpolate import interp1d
from scipy.fft import ifft
from diffpy.pdfgetx.cromermann import fxrayatq
%matplotlib qt


def getf0(element=None, qrange=np.array([])):
    f0 = [fxrayatq(element, qi) for qi in qrange]
    return f0


def getf2avg(data, dataformat,
             qmin, qmax, comp):
    x = data[:, 0]
    I = data[:, 1]
    
    if dataformat == 'tth':
        q_exp = 4*np.pi*np.sin(x/2*np.pi/180)/lamb
        q_trunc = q_exp[(q_exp >= qmin) & (q_exp <= qmax)]
        q_equi_step = (q_trunc[-1] - q_trunc[0])/(len(q_trunc)-1)
        q_equi_min = (q_trunc[0]//q_equi_step+1)*q_equi_step
        q_equi_max = (q_trunc[-1]//q_equi_step)*q_equi_step
        q_equi = np.linspace(q_equi_min, q_equi_max, len(q_trunc)-1)
    elif dataformat == 'q':
        q_equi = x[(x >= qmin) & (x <= qmax)]
        I_equi = I[(x >= qmin) & (x <= qmax)]
        q_equi_max = q_equi[-1]
    else:
        print('data format unrecognized')
    qstep = q_equi[1] - q_equi[0]
    qn_exp = int(np.round(q_equi[0]/qstep))
    elem_f0s = {}
    elem_frac = {}
    N = np.sum(np.array([n for n in comp.values()]))
    q_f2avg = np.linspace(0, q_equi_max, qn_exp+len(q_equi))
    f2avg = np.zeros_like(q_f2avg)
    for elem in comp.keys():
        elem_f0s[elem] = np.array(getf0(elem, q_f2avg))
        elem_frac[elem] = comp[elem]/N
        f2avg += elem_frac[elem]*elem_f0s[elem]**2
    return q_f2avg, f2avg


def getEDF(data, dataformat,
           comp, f2avg,
           lamb, qmin, qmax,
           rmin, rmax, rstep,
           scale, bkg):
    x = data[:, 0]
    I = data[:, 1]
    
    if dataformat == 'tth':
        q_exp = 4*np.pi*np.sin(x/2*np.pi/180)/lamb
        # Truncate and interpolate data in q
        q_trunc = q_exp[(q_exp >= qmin) & (q_exp <= qmax)]
        I_trunc = I[(q_exp >= qmin) & (q_exp <= qmax)]
        interpolate = interp1d(q_trunc, I_trunc, kind='linear')
        q_equi_step = (q_trunc[-1] - q_trunc[0])/(len(q_trunc)-1)
        q_equi_min = (q_trunc[0]//q_equi_step+1)*q_equi_step
        q_equi_max = (q_trunc[-1]//q_equi_step)*q_equi_step
        q_equi = np.linspace(q_equi_min, q_equi_max, len(q_trunc))
        I_equi = interpolate(q_equi)
    elif dataformat == 'q':
        q_equi = x[(x >= qmin) & (x <= qmax)]
        I_equi = I[(x >= qmin) & (x <= qmax)]
        q_equi_max = q_equi[-1]
    else:
        print('data format unrecognized')
    
    # Extent qgrid to hypothetical qmax based on rstep
    qstep = q_equi[1] - q_equi[0]
    rmax_qstep = np.pi/qstep
    rmax = min(rmax, rmax_qstep)
    qmax_hypo = np.pi/rstep
    qn_hypo = int(qmax_hypo/qstep)
    q_hypo = np.arange(0, qn_hypo)*qstep
    I_hypo = np.zeros_like(q_hypo)
    qn_exp = int(np.round(q_equi[0]/qstep))
    I_hypo[qn_exp:qn_exp+len(I_equi)] = scale*(I_equi - bkg)

    # Subtract self-scattering and multiply with q
    f2avg_hypo = np.zeros_like(q_hypo)
    f2avg_hypo[:len(f2avg)] = f2avg
    Fq_hypo = q_hypo*(I_hypo - f2avg_hypo)

    # Fourier transform
    lostep = int(np.ceil((rmin-1e-08)/rstep))
    histep = int(np.floor((rmax+1e-08)/rstep))
    rgrid = np.arange(lostep, histep)*rstep
    nin = len(q_hypo)
    nqmax_rstep = qmax_hypo/qstep
    nbase = max([nin, histep])
    nout_ = nqmax_rstep * (np.floor_divide(nbase, nqmax_rstep)+1)
    nout = int(np.round(nout_))
    if nout > 8 * nin:
        nlog2 = int(np.ceil(np.log2(nout)))
        nout = 2 ** nlog2
    qmaxdb = 2*nout*qstep
    yindb = np.zeros((2*nout), dtype=float)
    yindb[:nin] = Fq_hypo
    yindb[:-nin:-1] = -1 * Fq_hypo[1:]
    cyoutdb = ifft(yindb)
    xstepfine = 2*np.pi/qmaxdb
    xoutfine = np.arange(nout)*xstepfine
    youtfine = np.imag(cyoutdb[:nout])*(qmaxdb/np.pi)
    f = interp1d(xoutfine, youtfine, kind='linear')
    FT = f(rgrid)
    return rgrid, FT, q_hypo, Fq_hypo, I_hypo


# Load X-ray TS pattern
curdir = os.getcwd()
fname = '/Si_0p3_300K_BG_corrected.xy'
data = np.genfromtxt(curdir + fname)
dataformat = 'tth'        # 'q' or 'tth'
displayEDFslope = True    # True/False
displayf2avg = True       # True/False


# Experimental and computational detials
comp = {'C': 60}
volume = 5.431302**3
N = np.sum(np.array([n for n in comp.values()]))
Ne = np.sum(np.array([n*xraydb.atomic_number(ele) for ele, n in comp.items()]))
Ne_avg = Ne/N
avg_elec_dens = Ne/volume
avg_corr_dens = avg_elec_dens*Ne_avg
lamb = 0.4500
qmin = 1.0
qmax = 27
rmin = 0.0
rmax = 50.0
rstep = 0.01
scale = 0.01
bkg = 5000


# Calculate EDF
qf2avg, f2avg = getf2avg(data, dataformat,
                         qmin, qmax, comp)
r, EDF, q, Fq, Iq = getEDF(data, dataformat,
                           comp, f2avg,
                           lamb, qmin, qmax,
                           rmin, rmax, rstep,
                           scale, bkg)

# Interactive plot
autoscale = False

fig, axes = plt.subplots(3, 1, figsize=(10, 12))
plt.subplots_adjust(left=0.1, bottom=0.1, hspace=0.3)
axes = axes.ravel()
Iqfig = axes[0]
Fqfig = axes[1]
EDFfig = axes[2]

Iqfig.margins(x=0, y=0.5)
Fqfig.margins(x=0, y=0.5)
EDFfig.margins(x=0, y=0.5)

Iqfig.grid()
Fqfig.grid()
EDFfig.grid()

Iqplot, = Iqfig.plot(q, Iq, 'k.-', ms=1, lw=0.5)
Iqfig.set_xlabel('$q$')
Iqfig.set_ylabel('$I$($q$)')
Iqfig.set_xlim(0, qmax+2)
Iqfig.set_ylim(min(-0.25*max(f2avg), -0.25*max(abs(Iq))),
               max(1.25*max(f2avg), 1.25*max(abs(Iq))))
if displayf2avg:
    f2avgplot, = Iqfig.plot(qf2avg, f2avg, 'b-', label='$\\langle f^2 \\rangle$')
    Iqfig.legend()

Fqplot, = Fqfig.plot(q, Fq, 'k.-', ms=1, lw=0.5)
Fqfig.set_xlabel('$q$')
Fqfig.set_ylabel('$F$($q$)')
Fqfig.set_xlim(0, qmax+2)
Fqfig.set_ylim(-0.75*max(abs(Fq)),
               1.25*max(abs(Fq)))


EDFplot, = EDFfig.plot(r, EDF, 'k.-', ms=1, lw=0.5)
EDFfig.set_xlabel('$r$')
EDFfig.set_ylabel('EDF')
if displayEDFslope:
    EDFslopeplot, = EDFfig.plot(r, -4*np.pi*r*avg_corr_dens, 'b-', label='$-4\\pi r\\rho_{\\mathrm{ec,0}}$')
    EDFfig.legend()
EDFfig.set_xlim(0, rmax)
EDFfig.set_ylim(-0.75*max(abs(EDF[r>1])),
                1.25*max(abs(EDF[r>1])))

scale_slider_color = 'white'
scale_slider_specs = plt.axes([0.1, 0.94, 0.8, 0.03], facecolor=scale_slider_color)
scale_slider = Slider(ax=scale_slider_specs,
                      label='Scale',
                      valmin=0.0,
                      valmax=scale*2,
                      valstep=scale/200,
                      valinit=scale)

def update_scale_slider(val):
    r, EDF, q, Fq, Iq  = getEDF(data, dataformat,
                                comp, f2avg,
                                lamb, qmin, qmax,
                                rmin, rmax, rstep,
                                scale_slider.val, bkg_slider.val)
    Iqplot.set_ydata(Iq)
    Fqplot.set_ydata(Fq)
    EDFplot.set_ydata(EDF)
    if autoscale:
        Fqfig.set_ylim(-0.75*max(abs(Fq)),
                       1.25*max(abs(Fq)))
        EDFfig.set_ylim(-0.75*max(abs(EDF[r>1])),
                        1.25*max(abs(EDF[r>1])))
    fig.canvas.draw_idle()
scale_slider.on_changed(update_scale_slider)


bkg_slider_color = 'white'
bkg_slider_specs = plt.axes([0.1, 0.91, 0.8, 0.03], facecolor=bkg_slider_color)
bkg_slider = Slider(ax=bkg_slider_specs,
                    label='Const. bkg',
                    valmin=0,
                    valmax=2*abs(bkg),
                    valstep=bkg/500,
                    valinit=bkg)

def update_bkg_slider(val):
    r, EDF, q, Fq, Iq  = getEDF(data, dataformat,
                                comp, f2avg,
                                lamb, qmin, qmax,
                                rmin, rmax, rstep,
                                scale_slider.val, bkg_slider.val)
    Iqplot.set_ydata(Iq)
    Fqplot.set_ydata(Fq)
    EDFplot.set_ydata(EDF)
    if autoscale:
        Fqfig.set_ylim(-0.75*max(abs(Fq)),
                       1.25*max(abs(Fq)))
        EDFfig.set_ylim(-0.75*max(abs(EDF[r>1])),
                        1.25*max(abs(EDF[r>1])))
    fig.canvas.draw_idle()
bkg_slider.on_changed(update_bkg_slider)

plt.show()

In [71]:
scale = 0.0031
bkg = 4500
r, EDF, q, Fq, Iq = getEDF(data, dataformat,
                           comp, f2avg,
                           lamb, qmin, qmax,
                           rmin, rmax, rstep,
                           scale, bkg)

with open(curdir + fname[:-3] + '_EDF_v2.xy', "w") as f:
    for i, EDFi in enumerate(EDF):
        f.write('{:6f}\t{:6f}\n'.format(r[i], EDFi))
    
with open(curdir + fname[:-3] + '_EDF_v2_Fq.xy', "w") as f:
    for i, Fqi in enumerate(Fq[q<=qmax]):
        f.write('{:6f}\t{:6f}\n'.format(q[i], Fqi))

with open(curdir + fname[:-3] + '_EDF_v2_info.txt', "w") as f:
    f.write(f"composition = {comp}\n")
    f.write(f"qmin = {qmin:.6f}\n")
    f.write(f"qmax = {qmax:.6f}\n")
    f.write(f"rmin = {rmin:.6f}\n")
    f.write(f"rmax = {rmax:.6f}\n")
    f.write(f"rstep = {rstep:.6f}\n")
    f.write(f"scale = {float(scale):.6f}\n")
    f.write(f"const. bkg. = {float(bkg):.6f}\n")